## ANN dMap all-atom latent space -> pairwise distances in molecule

### setup

In [ ]:
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sys
import math
import time
import pickle
from copy import deepcopy

import numpy as np
import pandas as pd

import torch
from torch import nn #import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch.nn.parallel
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data

import sklearn.preprocessing as pre
from sklearn.model_selection import train_test_split

from collections import OrderedDict

import mdtraj as md
import nglview as nv

In [ ]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2'''

In [ ]:
if torch.cuda.is_available():
    print("=> Using GPU")
    print("CUDA device count =")
    print torch.cuda.device_count()
    print("Selecting decvice = cuda:0")
    device = torch.device("cuda:0")
    print("Device name = ")
    print torch.cuda.get_device_name(0)
else:
    print("=> Using CPU")
    device = torch.device("cpu")

print("Using device = %s" % device)

In [ ]:
# fix random seed for reproducibility
np.random.seed(200186)
torch.manual_seed(200186)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(200186)

### loading simulation trajectory

In [ ]:
trj_dir = "../C24/"
pdb_file = os.path.join(trj_dir, "after_pr_protein.pdb")
trj_file = os.path.join(trj_dir, "md_protein_fit.xtc")

In [ ]:
# loading
traj = md.load(trj_file, top=pdb_file)

In [ ]:
# rot+trans alignment of trajectory to first frame
traj.center_coordinates(mass_weighted=False)
traj.superpose(traj[0])

In [ ]:
print('How many frames? %s' % traj.n_frames)
print('How many atoms?    %s' % traj.n_atoms)
print(traj.xyz.shape)

In [ ]:
# visualizing
view = nv.show_mdtraj(traj)
view.remove_cartoon()
view.add_ball_and_stick()
view

### loading principal moments of gyration tensor

In [ ]:
npzfile = np.load('../0_traj2pdh2t/princMom.npz')
sorted(npzfile.files)
princMom = npzfile['princMom']
Rg = npzfile['Rg']

print(princMom.shape)
print(Rg.shape)

### loading h2t

In [ ]:
npzfile = np.load('../0_traj2pdh2t/h2t.npz')
sorted(npzfile.files)
h2t = npzfile['h2t']

print(len(h2t))

### loading all-atom dMap embedding

In [ ]:
# loading z_aa
npzfile = np.load('../1A_dMap_aa/data.npz')
sorted(npzfile.files)
z_aa = npzfile['z']

In [ ]:
# plotting all-atom dMap embedding

plot_stride = 50

colorMap = princMom[:z_aa.shape[0]:plot_stride,0]

fig = plt.figure()
ax = fig.add_subplot(111)
i=1
j=2
ax.scatter(z_aa[::plot_stride,i], z_aa[::plot_stride,j], c=colorMap)
ax.set_xlabel('psi_' + str(i+1))
ax.set_ylabel('psi_' + str(j+1))
plt.xlim([np.min(z_aa[:,i])*1.1,np.max(z_aa[:,i])*1.1])
plt.ylim([np.min(z_aa[:,j])*1.1,np.max(z_aa[:,j])*1.1])
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('z_aa_23.png', dpi=300)
plt.close()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
i=1
j=2
k=3
ax.scatter(z_aa[::plot_stride,i], z_aa[::plot_stride,j], z_aa[::plot_stride,k], c=colorMap)
ax.set_xlabel('psi_' + str(i+1))
ax.set_ylabel('psi_' + str(j+1))
ax.set_zlabel('psi_' + str(k+1))
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('z_aa_234.png', dpi=300)
plt.close()

### extracting all-atom dMap embedding features

In [ ]:
z_aa_features_idx = np.array([1,2])
z_aa = z_aa[:,z_aa_features_idx]

n_aa = z_aa.shape[0]
d_aa = z_aa.shape[1]

print('n_aa = %d' % n_aa)
print('d_aa = %d' % d_aa)

### scaling all-atom dMap embedding

In [ ]:
# scaling
zscale_aa = pre.MinMaxScaler(feature_range=(0,1)) # changing feature range requires modification of chain rule in post-hoc gradient rescaling AND use of loss other than BCE

Z_aa = zscale_aa.fit_transform(z_aa)

# saving scaling associated with model
D = {}
D['zscale'] = zscale_aa
file = open('ANN_z_aa_scaling.pkl', 'wb')
pickle.dump(D, file)

### loading pairwise distances

In [ ]:
npzfile = np.load('../0_traj2pdh2t/pd.npz')
sorted(npzfile.files)
pd = npzfile['pd']

In [ ]:
# no. of frames
nFrames = pd.shape[0]
print('nFrames = %d' % nFrames)

In [ ]:
# no. of atoms (between which pairwise distances are computed) and no. of pairs
Npairs = pd.shape[1]
print('Npairs = %d' % Npairs)

N = (1. + np.sqrt(1.+8.*Npairs))/2.
N = np.int(N)
print('N = %d' % N)

assert N*(N-1) == 2*Npairs

### scaling pairwise distances

In [ ]:
# scaling
zscale_pd = pre.MinMaxScaler(feature_range=(0,1)) # changing feature range requires modification of chain rule in post-hoc gradient rescaling AND use of loss other than BCE

Z_pd = zscale_pd.fit_transform(pd)

# saving scaling associated with model
D = {}
D['zscale'] = zscale_pd
file = open('ANN_pd_scaling.pkl', 'wb')
pickle.dump(D, file)
file.close()


### trimming {z_aa, Z_aa} and {pd, Z_pd}

In [ ]:
startFrame = 0 #training parameters for C24
stopFrame = 100000 
skipper = 10 

z_aa = z_aa[startFrame:stopFrame:skipper,:]
Z_aa = Z_aa[startFrame:stopFrame:skipper,:]
pd = pd[startFrame:stopFrame:skipper,:]
Z_pd = Z_pd[startFrame:stopFrame:skipper,:]

assert z_aa.shape[0] == Z_aa.shape[0]
assert z_aa.shape[0] == pd.shape[0]
assert z_aa.shape[0] == Z_pd.shape[0]

print(z_aa.shape[0])

### initializing ANN

In [ ]:
class ANN(nn.Module):
    def __init__(self, q, d_in, d_out):
        super(ANN, self).__init__()
        
        n_nodes = np.ceil(q*np.linspace(d_in,d_out,4)).astype(int) # e.g. d_in = 2, d_out = 276, q = 2 => n_nodes = [  4 187 370 552]
        
        self.hidden1 = nn.Linear(d_in, n_nodes[0])
        self.hidden2 = nn.Linear(n_nodes[0], n_nodes[1])
        self.hidden3 = nn.Linear(n_nodes[1], n_nodes[2])
        self.hidden4 = nn.Linear(n_nodes[2], n_nodes[3])
        self.output = nn.Linear(n_nodes[3], d_out)
        
    def forward(self, x):
        x = torch.tanh(self.hidden1(x))
        x = torch.tanh(self.hidden2(x))
        x = torch.tanh(self.hidden3(x))
        x = torch.tanh(self.hidden4(x))
        x = torch.tanh(self.output(x))
        return x

In [ ]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

In [ ]:
# https://discuss.pytorch.org/t/how-can-l-load-my-best-model-as-a-feature-extractor-evaluator/17254/5
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

In [ ]:
x_tensor = torch.from_numpy(Z_aa).float()
y_tensor = torch.from_numpy(Z_pd).float()

assert x_tensor.shape[0] == y_tensor.shape[0]

print(x_tensor.shape[0])

### training ANN

In [ ]:
idx = np.arange(x_tensor.shape[0])

val_frac = 0.10
idx_train, idx_val = train_test_split(idx, test_size=val_frac, random_state=42)

train_dataset = CustomDataset(x_tensor[idx_train], y_tensor[idx_train])
val_dataset = CustomDataset(x_tensor[idx_val], y_tensor[idx_val])

In [ ]:
q=2.

model = ANN(q, d_aa, Npairs).to(device)

save_path = "./ANN.pyt"
model.load_state_dict(torch.load(save_path, map_location=device))

lr = 1e-3
n_epochs = 150
batch_size = 500

loss_fn = nn.MSELoss(reduction='elementwise_mean')
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0)

train_step = make_train_step(model, loss_fn, optimizer)

n_batch = math.ceil(1. * len(train_dataset) / batch_size)

In [ ]:
loss_train = np.zeros(n_epochs)
loss_val = np.zeros(n_epochs)

start=time.time()

for epoch in range(n_epochs):    
    
    dataset = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    
    model.train()
    
    batch_idx = 0
    for train_in, train_out in dataset:
        train_in, train_out = train_in.to(device), train_out.to(device)
        out = model(train_in)
        loss = loss_fn(out, train_out) 
        optimizer.zero_grad()
        loss.backward() # scalar variables, when we call .backward() on them, don’t require arguments – only tensors require a matching sized tensor argument to be passed to the .backward() operation
        optimizer.step()
        batch_idx += 1

    model.eval()
    
    y_pred = model(train_dataset.x.to(device))
    diff = y_pred.cpu().detach().numpy() - train_dataset.y.cpu().detach().numpy()
    MSE = np.mean(diff**2)
    loss_train[epoch] = MSE
    
    y_pred = model(val_dataset.x.to(device))
    diff = y_pred.cpu().detach().numpy() - val_dataset.y.cpu().detach().numpy()
    MSE = np.mean(diff**2)
    loss_val[epoch] = MSE        
    
    if np.mod(epoch+1,1) == 0:
        print('====> Epoch %d of %d done! Train loss = %.2e, Val loss = %.2e' % (epoch+1,n_epochs,loss_train[epoch],loss_val[epoch]))

end = time.time()
print("Using device = %s" % device)
print("Elapsed time %.2f (s)" % (end - start))

In [ ]:
# plotting training and validation losses

fig, ax = plt.subplots()
ax.plot(np.arange(len(loss_train)), loss_train, 'b--')
ax.plot(np.arange(len(loss_val)), loss_val, 'r--')
ax.set_xlabel("epoch")
ax.set_ylabel("loss")
ax.set_yscale("log")
ax.legend(["train","val"])
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('ANN_training.png', dpi=300)
plt.close()

In [ ]:
# saving trained model
save_path = "./ANN.pyt"
torch.save(model.state_dict(), save_path)

### running ANN on all data

#### loading data

In [ ]:
# INPUTS

# loading z_aa
npzfile = np.load('../1A_dMap_aa/data.npz')
sorted(npzfile.files)
z_aa = npzfile['z']

# extracting features
z_aa = z_aa[:,z_aa_features_idx]

d_aa = z_aa.shape[1]

# loading z_aa scaling
D = pickle.load(open("ANN_z_aa_scaling.pkl", "rb"))
zscale_aa = D['zscale']

# scaling data
Z_aa = zscale_aa.transform(z_aa)

In [ ]:
# OUTPUTS

# loading pairwise distances
npzfile = np.load('../0_traj2pdh2t/pd.npz')
sorted(npzfile.files)
pd = npzfile['pd']

Npairs = pd.shape[1]

# loading pd scaling
D = pickle.load(open("ANN_pd_scaling.pkl", "rb"))
zscale_pd = D['zscale']

# scaling data
Z_pd = zscale_pd.transform(pd)

In [ ]:
# PORTING TO TORCH

x_tensor = torch.from_numpy(Z_aa).float()
y_tensor = torch.from_numpy(pd).float()

assert x_tensor.shape[0] == y_tensor.shape[0]
print(x_tensor.shape[0])

#### loading model

In [ ]:
model = ANN(q, d_aa, Npairs).to(device)

save_path = "./ANN.pyt"
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

#### running all data through model

In [ ]:
# initializing storage arrays
Z_pd_pred = np.zeros(shape=(0,Z_pd.shape[1]), dtype=float)

# running all data through model
# N.B. running as batches so as not to max out GPU memory
nSamples = Z_aa.shape[0]
maxBatchSz = 50000
assert nSamples >= maxBatchSz
nBatch = np.ceil(np.float(nSamples)/np.float(maxBatchSz)).astype(int)
batchEdges = np.floor(np.linspace(0,nSamples,nBatch+1)).astype(int)

print('Running data through trained network...')
for i in range(nBatch):
    data = torch.FloatTensor(Z_aa[batchEdges[i]:batchEdges[i+1],:])
    data = Variable(data, requires_grad=False)
    data = data.to(device)
    model.eval()
    pred = model(data)

    Z_pd_pred_i = pred.cpu().detach().numpy()

    Z_pd_pred = np.append(Z_pd_pred,Z_pd_pred_i,axis=0)

    print('\tProcessed batch %d/%d...' % (i+1,nBatch))

print('DONE!')
print('')

pd_pred = zscale_pd.inverse_transform(Z_pd_pred)

# scaled MSE
diff = Z_pd_pred - Z_pd
MSE = np.mean(diff**2)
print('MSE_test = %.2e' % MSE)
print('RMSE_test = %.2e' % np.sqrt(MSE))
print('')

# unscaled MSE
diff = pd_pred - pd
MSE = np.mean(diff**2)
print('MSE_test = %.2e nm' % MSE)
print('RMSE_test = %.2e nm' % np.sqrt(MSE))
print('')

# saving latent space projections
np.savez('ANN_pred.npz', pd_pred=pd_pred)